In [1]:
import pprint, pickle, shutil, os, itertools, ast, pandas as pd, numpy as np

from datetime import datetime

from distributions import *
from scipy.stats import geom, dlaplace, poisson, ttest_rel, ttest_ind, rankdata
from sklearn.preprocessing import LabelEncoder
from sklearn.dummy import DummyClassifier
from sklearn_classifiers import PRIOR_BEST_PARAMS
from sklearn.metrics import f1_score

import matplotlib, matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = (15, 15)
matplotlib.rcParams.update({'font.size': 22})

In [109]:
GCN_SERVERS = ['gray', 'nightingale', 'harrison']
SKLEARN_SERVERS = ['safar', 'gray', 'nightingale', 'harrison']
GPUS = range(4)
TRACKS = range(10)
BKP_DIR = '01-24-18_bkps'

DATA_DIR = './data/'
SEARCH_RESULTS_FILEPATH = os.path.join(DATA_DIR, 'merged_results.hdf')

pp = pprint.PrettyPrinter(indent=1)

In [110]:
gcn_GSE92742_results_1 = pd.read_hdf(SEARCH_RESULTS_FILEPATH, 'gcn')
gcn_GSE92742_results_2 = pd.read_hdf(SEARCH_RESULTS_FILEPATH, 'gcn_GSE92742')
sklearn_results = pd.read_hdf(SEARCH_RESULTS_FILEPATH, 'sklearn')
print(
    "Collected %d GSE92742 GSE results (%d erroneous), "
    "and %d Sklearn results (%d erroneous)" % (
        gcn_GSE92742_results_1.shape[0] + gcn_GSE92742_results_2.shape[0],
        (
            gcn_GSE92742_results_1[gcn_GSE92742_results_1['Error']].shape[0] +
            gcn_GSE92742_results_2[gcn_GSE92742_results_2['Error']].shape[0]
        ),
        sklearn_results.shape[0], sklearn_results[sklearn_results['Error']].shape[0],
    )
)

Collected 1600 GSE92742 GSE results (357 erroneous), and 14136 Sklearn results (710 erroneous)


# Deep Dives

# Valid Result Analysis

#### Majority Class Classifier Baselines

In [111]:
GSE92742 = pd.read_hdf(os.path.join(DATA_DIR, 'GSE92742_fully_restricted.hdf'))
GSE92742_prostate = pd.read_hdf(os.path.join(DATA_DIR, 'GSE92742_fully_restricted_prostate.hdf'))

In [112]:
CLASSIFIER_DATASETS = {
    'GSE92742': (
        GSE92742, 'Fold', [('Primary Site', 'primary_site'), ('Subtype', 'subtype'), ('MOA', 'moa')]
    ),
    'GSE92742 (Prostate Only)': (
        GSE92742_prostate, 'Fold', [('MOA', 'moa')]
    ),
}

In [113]:
for dataset_name, (d, fold_col, targets) in CLASSIFIER_DATASETS.items():
    folds = d.index.get_level_values(fold_col)
    print(dataset_name, d.shape)
    cells = d.index.get_level_values('cell_id')
    print('  %d cells total' % len(set(cells)))
    counts, max_cell, max_cnt, min_cell, min_cnt = {}, None, 0, None, float('inf')
    for c in list(cells.values):
        if c in counts: counts[c] += 1
        else: counts[c] = 1

    for c, cnt in counts.items():                            
        if cnt > max_cnt: max_cell, max_cnt = c, cnt
        if cnt < min_cnt: min_cell, min_cnt = c, cnt

    print('    Most Frequent Cell %s at %d' % (max_cell, max_cnt))
    print('    Least Frequent Cell %s at %d' % (min_cell, min_cnt))

    
    for target_name, target in targets:
        print('  %s' % target_name)
        K, accuracies, f1s, L, targets = folds.max(), [], [], LabelEncoder(), d.index.get_level_values(target)
        print('    %d Classes total' % len(set(targets)))
        L.fit(targets.values)
        
        counts, max_target, max_cnt, min_target, min_cnt = {}, None, 0, None, float('inf')
        for t in list(targets.values):
            if t in counts: counts[t] += 1
            else: counts[t] = 1

        for t, cnt in counts.items():                            
            if cnt > max_cnt: max_target, max_cnt = t, cnt
            if cnt < min_cnt: min_target, min_cnt = t, cnt
        
        print('    Most Frequent Class %s at %d' % (max_target, max_cnt))
        print('    Least Frequent Class %s at %d' % (min_target, min_cnt))
            
        for fold in range(K):
            dev_fold, test_fold = fold, (fold + 1) % K
            train_df = d[~folds.isin([dev_fold, test_fold])]
            dev_df = d[folds == dev_fold]
            # test_fold = dataset[folds == test_fold]
            C = DummyClassifier(strategy='prior')
            C.fit(train_df.values, L.transform(train_df.index.get_level_values(target)))
            labels = L.transform(dev_df.index.get_level_values(target))
            accuracies.append(C.score(dev_df.values, labels))
            f1s.append(f1_score(labels, C.predict(dev_df.values), average='macro'))
        accuracies = 100.0*np.array(accuracies)
        f1s = 100.0 * np.array(f1s)
        print(
            "    Majority Class Classifier: %.2f ± %.2f, %.2f ± %.4f" % (
                np.mean(accuracies), np.std(accuracies),
                np.mean(f1s), np.std(f1s)
            )
        )
            

GSE92742 (156461, 978)
  36 cells total
    Most Frequent Cell MCF7 at 26546
    Least Frequent Cell NCIH716 at 8
  Primary Site
    12 Classes total
    Most Frequent Class prostate at 43686
    Least Frequent Class ovary at 415


/scratch/conda_envs/gene_expression_modelling/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


    Majority Class Classifier: 27.89 ± 0.16, 3.63 ± 0.0158
  Subtype
    14 Classes total
    Most Frequent Class adenocarcinoma at 53245
    Least Frequent Class embryonal kidney at 1384
    Majority Class Classifier: 33.99 ± 0.21, 3.62 ± 0.0167
  MOA
    49 Classes total
    Most Frequent Class DMSO at 25638
    Least Frequent Class IKK inhibitor at 828
    Majority Class Classifier: 16.36 ± 0.16, 0.57 ± 0.0047
GSE92742 (Prostate Only) (25565, 978)
  2 cells total
    Most Frequent Cell PC3 at 13625
    Least Frequent Cell VCAP at 11940
  MOA
    9 Classes total
    Most Frequent Class DMSO at 8833
    Least Frequent Class Serotonin receptor antagonist at 1029
    Majority Class Classifier: 34.54 ± 0.05, 5.71 ± 0.0058


#### Learned Model Results

In [114]:
time_fill_value = 'Pre 01/11/18 @ 17:28'
sentinel_time = '2018-02-06T17:46:26.439047'

In [122]:
ERROR_COLS = ['Error', 'Error Message', 'Error Args', 'Error Type']
FUNCTIONS = {'mean': np.mean, 'std': np.std, 'min': np.min, 'max': np.max, 'to_list': lambda x: x}
def flatten(l): return [item for sublist in l for item in sublist]
def to_list(x): return str(x.tolist())
def str_to_list(x, t): return [t(e) for e in x[1:-1].split(', ')]
def split_row(df, r, start, end, K):
    row = df.iloc[r]
    if (
        row['Test Fold']['to_list'][start:end] != list(range(1, end - start)) + [(end-start) % K] and 
        row['Test Fold']['to_list'][start:end] != list(range(1, end - start + 1)) 
    ):
        print("Error: Test fold list doesn't look right!")
        print(row['Test Fold']['to_list'], start, end, K)
        display(row)
        raise NotImplementedError
        
    new_data = {
        (column, level): [FUNCTIONS[level](
            row[column]['to_list'][start:end]
        )] for column, level in list(itertools.product(
            ['Test Accuracy', 'Test F1', 'Dev Accuracy', 'Dev F1'], ['mean', 'std', 'to_list']
        )) + [('Wall Time', 'min'), ('Wall Time', 'max'), ('Wall Time', 'to_list')]
    }
    try:
        new_df = pd.DataFrame(data=new_data, index=df.iloc[r:r+1].index)
    except Exception as e:
        print(new_data)
        raise e
    new_df.columns = pd.MultiIndex.from_tuples(
        [(x,) if type(x) is not tuple else x for x in list(new_df.columns.values)]
    )
    new_df['Sample Count'] = end - start
    return new_df

def build_valid_results(df, sentinel_time='2018-02-06T17:46:26.439047'):
    running = df[
        (~df.Error) & (df['Wall Time'] != time_fill_value)
    ].drop(
        columns=ERROR_COLS + ['Train Time', 'Dev Fold', 'Dev Fold Size']
    )
    if sentinel_time is not None:
        old     = running[running['Wall Time'] <= sentinel_time]
        running = running[running['Wall Time'] > sentinel_time]
    else: old=None
        
    return pd.concat((process_results(running), process_results(old))) if old is not None else None

def process_results(running):
    if 'Classifer Name' in running.columns:
        running = running.drop(columns=['Classifier Name']).rename(columns={'Classifer Name': 'Classifier Name'})
        running['Graph Name'] = running['Graph Name'].fillna('N/A')
        running['Graph Edge Weight Cutoff'] = running['Graph Edge Weight Cutoff'].fillna(-1.0)
    else:
        running['Classifier Name'] = 'Graph Convolutional Neural Network'
        KEYS_TO_DROP = ['dir_name', 'eval_frequency', 'brelu', 'filter']
        running['Params'] = running['Params'].apply(
            lambda x: str({k: v for k, v in ast.literal_eval(x).items() if k not in KEYS_TO_DROP})
        )
        
    running['Test Fold'] = running['Test Fold'].astype(int)
    
    running = running.groupby(
        ['Dataset Name', 'Classifier Name', 'Target Name', 'Params', 'Graph Name', 'Graph Edge Weight Cutoff']
    ).agg({
        'Dev Accuracy': ['mean', 'std', 'count', to_list],
        'Dev F1': ['mean', 'std', to_list],
        'Test Accuracy': ['mean', 'std', to_list],
        'Test F1': ['mean', 'std', to_list],
#         'RNA Seq Accuracy': ['mean', 'std', to_list],
#         'RNA Seq F1':  ['mean', 'std', to_list],
        'Wall Time': ['min', 'max', to_list],
        'Test Fold': [to_list],
    })
        
    running['Sample Count'] = running['Dev Accuracy']['count']
    running = running.drop(columns=[('Dev Accuracy', 'count')])
    
    columns = ['Sample Count', 'Test Accuracy', 'Test F1', 'Dev Accuracy', 'Dev F1', 'Wall Time', 'Test Fold']
    running = running[columns]
    for column in columns[1:]:
        if column == 'Test Fold': fn = lambda x: str_to_list(x, int)
        elif column == 'Wall Time': fn = lambda x: str_to_list(x, lambda e: datetime.strptime(
            e[1:e.rfind('.')], "%Y-%m-%dT%H:%M:%S"
        ))
        else: fn = lambda x: str_to_list(x, float)
        running.loc[:, (column, 'to_list')] = running[column]['to_list'].apply(fn)
        
    desired_sample_count = pd.Series(
        running.index.get_level_values('Dataset Name').values, index=running.index
    ).apply(lambda x: 15 if 'CEGS001' in x else 10)
    
    over_counted_idx = running['Sample Count'] > desired_sample_count
    over_counted, lteq_counted = running[over_counted_idx], running[~over_counted_idx]
    desired_over = desired_sample_count[over_counted_idx]
    lteq_counted['Run Number'] = 1
    
    over_counted_split = []
    for i in range(len(over_counted)):
        start, run, K, N = 0, 1, desired_over.iloc[i], len(over_counted.iloc[i]['Test Fold']['to_list'])
        while start < N:
            end = min(N, start + K)
            new_df = split_row(over_counted, i, start, end, K)
            new_df['Run Number'] = run
            over_counted_split.append(new_df)
            
            start = end
            run += 1
    
    running = pd.concat([lteq_counted] + over_counted_split) if len(over_counted_split) > 0 else lteq_counted
        
    desired_sample_count = pd.Series(
        running.index.get_level_values('Dataset Name').values, index=running.index
    ).apply(lambda x: 15 if 'CEGS001' in x else 10)

    running['Incomplete'] = running['Sample Count'] < desired_sample_count
    running.set_index(['Run Number', 'Incomplete'], append=True, inplace=True)
    running = running[columns].sort_values(by=('Dev Accuracy', 'mean'), axis='index', ascending=False)
        
    return running


In [131]:
sklearn_processed = build_valid_results(sklearn_results)
GSE92742_processed = pd.concat((
    build_valid_results(gcn_GSE92742_results_2), build_valid_results(gcn_GSE92742_results_1)
))

all_results = [('GSE92742', GSE92742_processed), ('SKlearn', sklearn_processed)]

/scratch/conda_envs/gene_expression_modelling/lib/python3.5/site-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Counts & Maxes

In [124]:
GRAPHS = {'GSE92742': ['regnetworkdb', 'prostate_humanbase']}
CLASSIFIERS = [
    'Decision Tree', 'K-neighbors', 'Linear Classifier', 'Random Forest', 'SKlearn Fully Connected ANN'
]

### Counts

In [125]:
for name, df in all_results:
    print(name)
    if name != 'SKlearn': print(df.shape[0])
    if name == 'SKlearn':
        df_grouped = df[('Dev Accuracy', 'mean')].groupby('Classifier Name').agg(['count'])
        with pd.option_context('display.max_rows', None):
            display(df_grouped)

GSE92742
140
SKlearn


,count
Classifier Name,
AdaBoost,6
Decision Tree,261
K-neighbors,68
Kenerl SVC,7
Linear Classifier,450
Naive Bayes,1
Quadratic Discriminant Analysis,5
Random Forest,197
SKlearn Fully Connected ANN,385


In [126]:
for name, df in all_results:
    print(name)
    groups = ['Dataset Name', 'Target Name']
    if name == 'GSE92742':
        groups.append('Graph Name')
        df = df[df.index.get_level_values('Graph Name').isin(GRAPHS[name])]
    else:
        groups.append('Classifier Name')
    df_grouped = df[('Dev Accuracy', 'mean')].groupby(groups).agg(['count'])
    with pd.option_context('display.max_rows', None):
        display(df_grouped)

GSE92742


count
Dataset Name             Target Name  Graph Name               
GSE92742                 MOA          regnetworkdb           20
                         Primary Site regnetworkdb           17
                         Subtype      prostate_humanbase      1
                                      regnetworkdb           14
GSE92742 (prostate only) Prostate MOA prostate_humanbase     36
                                      regnetworkdb           18

SKlearn


count
Dataset Name             Target Name  Classifier Name                       
GSE92742                 MOA          AdaBoost                             1
                                      Decision Tree                       70
                                      K-neighbors                         18
                                      Kenerl SVC                           2
                                      Linear Classifier                  115
                                      Quadratic Discriminant Analysis      1
                                      Random Forest                       48
                                      SKlearn Fully Connected ANN        100
                         Primary Site Decision Tree                       70
                                      K-neighbors                         14
                                      Kenerl SVC                           2
                                      Linear Classifier                  108
                                      Quadratic Discriminant Analysis      1
                                      Random Forest                       49
                                      SKlearn Fully Connected ANN         80
                         Subtype      AdaBoost                             2
                                      Decision Tree                       59
                                      K-neighbors                         13
                                      Linear Classifier                  109
                                      Quadratic Discriminant Analysis      2
                                      Random Forest                       37
                                      SKlearn Fully Connected ANN         99
GSE92742 (prostate only) Prostate MOA AdaBoost                             3
                                      Decision Tree                       62
                                      K-neighbors                         23
                                      Kenerl SVC                           3
                                      Linear Classifier                  118
                                      Naive Bayes                          1
                                      Quadratic Discriminant Analysis      1
                                      Random Forest                       63
                                      SKlearn Fully Connected ANN        106

### Maxes

In [127]:
def display_with_test(df, n, to_print, add_test=True):
    if df.shape[0] == 0: return
    print(', '.join(to_print))
    d = df.head(n)
    if add_test and d[('Test Accuracy', 'mean')].isnull().all():
        d2 = df[~df[('Test Accuracy', 'mean')].isnull()].head(1)
        d = pd.concat((d, d2))
    
    return d

def display_maxes(df, n, skip_datasets=[]):
    datasets, targets = [df.index.get_level_values(l) for l in ['Dataset Name', 'Target Name']]
    for dataset in set(datasets):
        if dataset in skip_datasets: continue
        for target in set(targets):
            if target == 'Diagnosis': continue
            d = df[(datasets == dataset) & (targets == target)]
            if 'Classifier Name' in d.index.names:
                for classifier in CLASSIFIERS:
                    d2 = d[d.index.get_level_values('Classifier Name') == classifier]
                    display(display_with_test(d2, n, (dataset, target, classifier)))
            else: display(display_with_test(d, n, (dataset, target)))
                
def all_maxes(dfs, n, add_test=True):
    all_targets = set()
    return_dfs = {}
    for name, df in dfs: all_targets.update(df.index.get_level_values('Target Name').values)
    for target in all_targets:
        if target == 'Diagnosis': continue
        print(target)
        all_to_display = []
        for name, df_full in dfs:
            df = df_full[df_full.index.get_level_values('Incomplete') == False]
            datasets, targets = [df.index.get_level_values(l) for l in ['Dataset Name', 'Target Name']]
            for dataset in set(datasets):
                d = df[(datasets == dataset) & (targets == target)].copy()
                if d.shape[0] == 0: 
                    print("No data for", name, dataset, target)
                    continue
                for classifier in set(df.index.get_level_values('Classifier Name')):
                    d2 = d[d.index.get_level_values('Classifier Name') == classifier]
                    all_to_display.append(display_with_test(d2, n, [], add_test))
        all_df = pd.concat(all_to_display).sort_values(
            by=('Dev Accuracy', 'mean'), axis='index', ascending=False
        )[[
                'Sample Count', 'Test Accuracy', 'Test F1', 'Dev Accuracy', 'Dev F1', 'Wall Time',
#                 'RNA Seq Accuracy', 'RNA Seq F1'
            ]]
        return_dfs[target] = all_df
        display(all_df)
        
    return return_dfs

In [133]:
optimal_dfs = all_maxes(all_results, 1, False)

MOA

No data for GSE92742 GSE92742 (prostate only) MOA








No data for SKlearn GSE92742 (prostate only) MOA


Sample Count  \
                                                                                                                                                                                          
Dataset Name Classifier Name                    Target Name Params                                             Graph Name   Graph Edge Weight Cutoff Run Number Incomplete                
GSE92742     Graph Convolutional Neural Network MOA         {'regularization': 0.01087902135723254, 'num_ep... regnetworkdb  0.0                     1          False                10   
             SKlearn Fully Connected ANN        MOA         {'beta_1': 0.919473939313036, 'activation': 're... N/A          -1.0                     1          False                10   
             K-neighbors                        MOA         {'p': 1, 'weights': 'distance', 'metric': 'canb... N/A          -1.0                     1          False                10   
             Linear Classifier                  MOA         {'random_state': 101, 'penalty': 'l1', 'l1_rati... N/A          -1.0                     1          False                10   
             Random Forest                      MOA         {'max_depth': 100, 'max_leaf_nodes': None, 'cri... N/A          -1.0                     1          False                10   
             Decision Tree                      MOA         {'max_features': None, 'criterion': 'entropy', ... N/A          -1.0                     1          False                10   
             Quadratic Discriminant Analysis    MOA         {'reg_param': 0.29502875285498964}                 N/A          -1.0                     1          False                10   
             Kenerl SVC                         MOA         {'decision_function_shape': 'ovr', 'random_stat... N/A          -1.0                     2          False                10   
             AdaBoost                           MOA         {'n_estimators': 49, 'learning_rate': 0.0019380... N/A          -1.0                     1          False                10   

                                                                                                                                                                           Test Accuracy  \
                                                                                                                                                                                    mean   
Dataset Name Classifier Name                    Target Name Params                                             Graph Name   Graph Edge Weight Cutoff Run Number Incomplete                 
GSE92742     Graph Convolutional Neural Network MOA         {'regularization': 0.01087902135723254, 'num_ep... regnetworkdb  0.0                     1          False          46.390154   
             SKlearn Fully Connected ANN        MOA         {'beta_1': 0.919473939313036, 'activation': 're... N/A          -1.0                     1          False          46.194575   
             K-neighbors                        MOA         {'p': 1, 'weights': 'distance', 'metric': 'canb... N/A          -1.0                     1          False          43.519347   
             Linear Classifier                  MOA         {'random_state': 101, 'penalty': 'l1', 'l1_rati... N/A          -1.0                     1          False          39.139788   
             Random Forest                      MOA         {'max_depth': 100, 'max_leaf_nodes': None, 'cri... N/A          -1.0                     1          False          37.773578   
             Decision Tree                      MOA         {'max_features': None, 'criterion': 'entropy', ... N/A          -1.0                     1          False          28.693116   
             Quadratic Discriminant Analysis    MOA         {'reg_param': 0.29502875285498964}                 N/A          -1.0                     1          False                NaN   
             Kenerl SVC                       

Prostate MOA
No data for GSE92742 GSE92742 Prostate MOA

No data for SKlearn GSE92742 Prostate MOA











Sample Count  \
                                                                                                                                                                                                       
Dataset Name             Classifier Name                    Target Name  Params                                             Graph Name   Graph Edge Weight Cutoff Run Number Incomplete                
GSE92742 (prostate only) SKlearn Fully Connected ANN        Prostate MOA {'learning_rate': 'invscaling', 'nesterovs_mome... N/A          -1.0                     1          False                10   
                         Graph Convolutional Neural Network Prostate MOA {'regularization': 0.004, 'num_epochs': 200, 'F... regnetworkdb  0.0                     1          False                10   
                         K-neighbors                        Prostate MOA {'metric': 'canberra', 'p': 12, 'weights': 'dis... N/A          -1.0                     1          False                10   
                         Kenerl SVC                         Prostate MOA {'C': 0.087612415524362011, 'kernel': 'poly', '... N/A          -1.0                     1          False                10   
                         Linear Classifier                  Prostate MOA {'eta0': 0.00031685190815167104, 'l1_ratio': 0.... N/A          -1.0                     1          False                10   
                         Random Forest                      Prostate MOA {'min_samples_split': 2, 'criterion': 'entropy'... N/A          -1.0                     1          False                10   
                         Decision Tree                      Prostate MOA {'min_impurity_decrease': 0.0014646397820345471... N/A          -1.0                     1          False                10   
                         Quadratic Discriminant Analysis    Prostate MOA {'reg_param': 0.0032986422926944399}               N/A          -1.0                     1          False                10   
                         AdaBoost                           Prostate MOA {'n_estimators': 51, 'learning_rate': 0.0032276... N/A          -1.0                     1          False                10   
                         Naive Bayes                        Prostate MOA {}                                                 N/A          -1.0                     1          False                10   

                                                                                                                                                                                        Test Accuracy  \
                                                                                                                                                                                                 mean   
Dataset Name             Classifier Name                    Target Name  Params                                             Graph Name   Graph Edge Weight Cutoff Run Number Incomplete                 
GSE92742 (prostate only) SKlearn Fully Connected ANN        Prostate MOA {'learning_rate': 'invscaling', 'nesterovs_mome... N/A          -1.0                     1          False          68.261284   
                         Graph Convolutional Neural Network Prostate MOA {'regularization': 0.004, 'num_epochs': 200, 'F... regnetworkdb  0.0                     1          False          67.713897   
                         K-neighbors                        Prostate MOA {'metric': 'canberra', 'p': 12, 'weights': 'dis... N/A          -1.0                     1          False          66.511732   
                         Kenerl SVC                         Prostate MOA {'C': 0.087612415524362011, 'kernel': 'poly', '... N/A          -1.0                     1          False                NaN   
                         Linear Classifier                  Prostate MOA {'eta0': 0.00031685190815167104, 'l1_ratio': 0.... N/A          -1.0                     1          Fal

Subtype

No data for GSE92742 GSE92742 (prostate only) Subtype







No data for SKlearn GSE92742 (prostate only) Subtype


Sample Count  \
                                                                                                                                                                                          
Dataset Name Classifier Name                    Target Name Params                                             Graph Name   Graph Edge Weight Cutoff Run Number Incomplete                
GSE92742     Graph Convolutional Neural Network Subtype     {'regularization': 0.005416198768028686, 'num_e... regnetworkdb  0.0                     1          False                10   
             SKlearn Fully Connected ANN        Subtype     {'activation': 'relu', 'nesterovs_momentum': Tr... N/A          -1.0                     1          False                10   
             K-neighbors                        Subtype     {'metric': 'canberra', 'p': 1, 'weights': 'unif... N/A          -1.0                     1          False                10   
             Quadratic Discriminant Analysis    Subtype     {'reg_param': 0.074721156855557344}                N/A          -1.0                     1          False                10   
             Linear Classifier                  Subtype     {'learning_rate': 'invscaling', 'alpha': 0.3629... N/A          -1.0                     1          False                10   
             Random Forest                      Subtype     {'criterion': 'gini', 'max_depth': 25, 'min_sam... N/A          -1.0                     1          False                10   
             AdaBoost                           Subtype     {'learning_rate': 0.0019536043114476128, 'n_est... N/A          -1.0                     1          False                10   
             Decision Tree                      Subtype     {'min_impurity_decrease': 3.188205651608861e-05... N/A          -1.0                     1          False                10   

                                                                                                                                                                           Test Accuracy  \
                                                                                                                                                                                    mean   
Dataset Name Classifier Name                    Target Name Params                                             Graph Name   Graph Edge Weight Cutoff Run Number Incomplete                 
GSE92742     Graph Convolutional Neural Network Subtype     {'regularization': 0.005416198768028686, 'num_e... regnetworkdb  0.0                     1          False          93.506417   
             SKlearn Fully Connected ANN        Subtype     {'activation': 'relu', 'nesterovs_momentum': Tr... N/A          -1.0                     1          False          90.492351   
             K-neighbors                        Subtype     {'metric': 'canberra', 'p': 1, 'weights': 'unif... N/A          -1.0                     1          False          89.782852   
             Quadratic Discriminant Analysis    Subtype     {'reg_param': 0.074721156855557344}                N/A          -1.0                     1          False                NaN   
             Linear Classifier                  Subtype     {'learning_rate': 'invscaling', 'alpha': 0.3629... N/A          -1.0                     1          False          62.553808   
             Random Forest                      Subtype     {'criterion': 'gini', 'max_depth': 25, 'min_sam... N/A          -1.0                     1          False          51.736809   
             AdaBoost                           Subtype     {'learning_rate': 0.0019536043114476128, 'n_est... N/A          -1.0                     1          False                NaN   
             Decision Tree                      Subtype     {'min_impurity_decrease': 3.188205651608861e-05... N/A          -1.0                     1          False          41.137319   

                                            

Primary Site

No data for GSE92742 GSE92742 (prostate only) Primary Site







No data for SKlearn GSE92742 (prostate only) Primary Site


Sample Count  \
                                                                                                                                                                                           
Dataset Name Classifier Name                    Target Name  Params                                             Graph Name   Graph Edge Weight Cutoff Run Number Incomplete                
GSE92742     Graph Convolutional Neural Network Primary Site {'regularization': 0.0030802351761877, 'num_epo... regnetworkdb  0.0                     1          False                10   
             SKlearn Fully Connected ANN        Primary Site {'learning_rate_init': 0.0553037462469582, 'act... N/A          -1.0                     1          False                10   
             K-neighbors                        Primary Site {'n_neighbors': 11, 'p': 1, 'metric': 'canberra... N/A          -1.0                     1          False                10   
             Kenerl SVC                         Primary Site {'degree': 2, 'decision_function_shape': 'ovr',... N/A          -1.0                     1          False                10   
             Quadratic Discriminant Analysis    Primary Site {'reg_param': 0.00026125663902671215}              N/A          -1.0                     1          False                10   
             Linear Classifier                  Primary Site {'learning_rate': 'invscaling', 'l1_ratio': 0.7... N/A          -1.0                     1          False                10   
             Random Forest                      Primary Site {'criterion': 'entropy', 'max_depth': 100, 'min... N/A          -1.0                     1          False                10   
             Decision Tree                      Primary Site {'max_depth': 100, 'min_samples_leaf': 1, 'min_... N/A          -1.0                     1          False                10   

                                                                                                                                                                            Test Accuracy  \
                                                                                                                                                                                     mean   
Dataset Name Classifier Name                    Target Name  Params                                             Graph Name   Graph Edge Weight Cutoff Run Number Incomplete                 
GSE92742     Graph Convolutional Neural Network Primary Site {'regularization': 0.0030802351761877, 'num_epo... regnetworkdb  0.0                     1          False          93.918602   
             SKlearn Fully Connected ANN        Primary Site {'learning_rate_init': 0.0553037462469582, 'act... N/A          -1.0                     1          False          90.581805   
             K-neighbors                        Primary Site {'n_neighbors': 11, 'p': 1, 'metric': 'canberra... N/A          -1.0                     1          False          89.556767   
             Kenerl SVC                         Primary Site {'degree': 2, 'decision_function_shape': 'ovr',... N/A          -1.0                     1          False                NaN   
             Quadratic Discriminant Analysis    Primary Site {'reg_param': 0.00026125663902671215}              N/A          -1.0                     1          False                NaN   
             Linear Classifier                  Primary Site {'learning_rate': 'invscaling', 'l1_ratio': 0.7... N/A          -1.0                     1          False          60.937529   
             Random Forest                      Primary Site {'criterion': 'entropy', 'max_depth': 100, 'min... N/A          -1.0                     1          False          57.157304   
             Decision Tree                      Primary Site {'max_depth': 100, 'min_samples_leaf': 1, 'min_... N/A          -1.0                     1          False          44.436211   

                       